### Import packages

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException

In [2]:
b_tech_link = "https://btech.com/en/laptop/laptops-1.html"
state_filepath = r"E:\other\Epsilon_Ai\Final Project\final project\B_TECH.pkl"
links = []

# open chromedriver on your PC

def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions# Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
    
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
    
    return web


# scroll down all the page to load all data of item                                   
def scroll_down_all_the_way():
    
    from time import sleep
    
    global scroll_delay_s
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'): # The scroller has reached the end
        start = end
        end = start + 1000
        # step by step
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(2)

# to show more items
def click_button_more_option(n=50):
    time.sleep(2)
    time.sleep(2)
    bage_hight = web.execute_script('return document.body.scrollHeight')
    # to mask sure we click button
    web.execute_script(f'window.scrollTo(1000,"{bage_hight-2000}")')
    
    
    for i in range(7, n, 2):
        time.sleep(2)
        try:
            show_more = web.find_element(by="xpath", value=f"/html/body/div[5]/main/div/div[1]/section[1]/div/div[{i}]")
            show_more.click()    # click item                           

        except:
            try:
                time.sleep(2)
                show_more.click() 
            except:
                break
    scroll_down_all_the_way()
    
    
# get all links of laptops        
def get_laptops_links():
    
    soup = BeautifulSoup(web.page_source, 'html.parser')
    global listing
    
    tag = soup.find_all("div", class_="product-item-view")
    global links
    
    for link in tag:
        try:
            links.append(link.a["href"])
        except:
            pass
    
    return links


# scrape data and append it in list of dictionary
def get_laptops_info(link):
    time.sleep(3)
    web.get(link)
    listing = {}
    
    listing["link"] = link
    global fidelity_sector_report
    fidelity_sector_report = dict()
    soup = BeautifulSoup(web.page_source, 'html.parser') # use BeautifulSoup to send request
    

    listing["title"] = soup.find("div", class_="page-title-wrapper product").get_text(strip=True)
    listing["price"] = soup.find("p", class_="price-huge-static").get_text(strip=True)
    
     # to save data in dectionary   
           
    tag = soup.find("div", id="product-overview-description").find("div", class_="details")
    if tag:                    
        feature = tag.find_all("div", class_="detail")
        global Keys
        global values
        
        try:
            Keys = [Key.strong.get_text(strip=True) for Key in feature] # keys of dictinary
            values = [value.p.get_text(strip=True) for value in feature] # values of dictinary
        except:
            real_len = len(Keys)
            Keys = [Key.strong.get_text(strip=True) for Key in feature[:real_len]]
            values = [value.p.get_text(strip=True) for value in feature[:real_len]]
        
        dict_feat = dict(zip(Keys, values))
        listing.update(dict_feat)

    global info   
    info = soup.find("div", class_="additional-attributes-wrapper table-wrapper").table.find_all("tr")
    if info:
        more_info = [i.get_text(strip=True) for i in info]
        listing['more_info'] = " ; ".join(more_info)

    print(link)
    print(f"{len(listing)} - ", listing)
    print("-" *100, "\n\n")

    return listing



# to save all operation on PC
def save_state():
    
    import joblib
    
    global listings
    global state_filepath

    
    state = listings
    joblib.dump(state, state_filepath)

In [6]:
listings = []
web = open_chromedriver()
web.get(b_tech_link)       
click_button_more_option()
links = get_laptops_links()

# to get all pages of website
while links:
    link = links.pop()
    try:
        listing = get_laptops_info(link)
        listings.append(listing)
        
        
    except :
            print('timeout detected... cooling off...')
            time.sleep(60.0)
            listing = get_laptops_info(link)
            listings.append(listing)
            

save_state()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_15788/3935002278.py:17: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


https://btech.com/en/microsoft-surfacego-mp39.html
13 -  {'link': 'https://btech.com/en/microsoft-surfacego-mp39.html', 'title': 'Microsoft Surface Go Laptop, Intel Core i5-1035G1, 12.4 Inch, 128GB SSD, 8GB RAM, Intel UHD Graphics, Windows 10 - PlatinumModel:Surface GoSKU:114628069010', 'price': '13,833', 'Brand': 'Microsoft', 'Model': 'Surface Go', 'Display Size': '12.4 Inch (1536x1024)', 'Processor': 'Intel Core i5-1035G1 - Quad Core', 'Storage': '128GB SSD', 'RAM': '8GB RAM LPDDR4X', 'Graphics Card Type': 'Intel UHD Graphics', 'Operating System': 'Windows 10', 'Color': 'Platinum', 'more_info': 'BrandMicrosoft ; Model NameSurface Go ; Display Size12.4 Inch ; Hard Capacity128 GB ; Hard Drive TypeSolid State Drive ; Processor ManufacturerIntel ; Processor TypeIntel Core i5-1035G1 ; Number of Processor CoreQuad Core ; Processor Speed1.0 GHz ; Processor SpeedUp to 3.6 GHz ; RAM8 GB ; Ram TypeLPDDR4X ; Operating System TypeWindows 10 ; Number of USB Ports1 x USB-C - 1 x USB-A ; USB Ports2

In [7]:
df = pd.DataFrame.from_records(listings)
df

,link,title,price,Brand,Model,Processor,Hard Disk Drive,RAM,Operating System,Color,...,الرام,كارت الشاشة (الجرافيكس),Ram,سعة الهارد ديسك,Model Name,iBrand,Attachments,نوع كارت الشاشة (الجرافيكس),كارت الشاشة,السعة الداخلية
0,https://btech.com/en/msi-gf65-laptop-intel-i7-...,"MSI GF65 Laptop, Intel Core i7-10750H 5.0 GHz,...","32,399",MSI,GF65,Intel Core i7-10750H 5.0 GHz,512GB SSD,8GB DDR4,Windows 10,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://btech.com/en/msi-gf65-laptop-intel-i7-...,"MSI GF65 Laptop, Intel core i7-10750H 5.0 GHz,...","33,749",MSI,GF65,Intel core i7-10750H 5.0 GHz,512GB SSD,16GB DDR4,Windows 10,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://btech.com/en/lenovo-legion-7-laptop-in...,"Lenovo Legion 7 Laptop, Intel Core i7-10750HH ...","43,065",Lenovo,Legion 7,Intel Core i7-10750HH 5.0 GHz,1TB SSD,24GB DDR4,Windows 10,Grey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://btech.com/en/hp-omen-15-laptop-intel-i...,"HP Omen 15 Laptop, Intel Core i7-10870H 2.2 GH...","49,275",HP,Omen 15,Intel Core i7-10870H 2.2 GHz,1TB SSD,16GB DDR4,Windows 10,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://btech.com/en/huawei-matebook-x-pro-202...,"Huawei MateBook X Pro 2021 Laptop, Intel Core ...","36,999",Huawei,MateBook X Pro 2021,Intel Core i7-1165G7,NaN,16 GB LPDDR4x 4266MHz,Windows 10,Space Grey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,https://btech.com/en/huawei-matebook-d-14-2020...,"Huawei MateBook D 14 Laptop, Intel Core i5-113...","17,199",Huawei,MateBook D 14,Intel Core i5-1135G7,NaN,8GB DDR4,Windows 10,Silver,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,https://btech.com/en/huawei-matebook-d-15-i5-1...,"Huawei MateBook D 15 Laptop, Intel Core i5-113...","14,899",Huawei,MateBook D 15,Intel Core i5-1135G7,NaN,8GB DDR4,Windows 10,Silver,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,https://btech.com/en/dell-g15-5510-i5-10500h-2...,"Dell G15-5510 Gaming Laptop, Intel Core i5-105...","16,099",Dell,G15-5510,Intel Core i5-10500H,NaN,8GB DDR4,FreeDos,Dark Shadow Grey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,https://btech.com/en/hp-290-g4-pc-i7-10700-1tb...,"HP 290 G4 Microtower 44F22ES PC, Intel Core i7...","15,222",HP,290 G4 Microtower PC,Intel Core i7-10700,NaN,8GB DDR4-2666 RAM,FreeDos,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
